# Using TensorFlow like NumPy

In [12]:
import numpy as np
import tensorflow as tf

## Tensors and Operations

To create a tensor we use `tf.constant()`.

In [3]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42) # escalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

`tf.Tensor` has a shape and a data type (dtype).

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(f'Shape: {t.shape}')
print(f'Data Type: {t.dtype}')

Shape: (2, 3)
Data Type: <dtype: 'float32'>


Indexing works much like in Numpy.

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

All sorts of tensor operations are available.

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t) # @ is for matrix multiplication.

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

## Tensors and NumPy

Create a Tensor from a NumPy array

In [13]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

Create a NumPy array from a Tensor

In [14]:
t.numpy() # or np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

Apply TensorFlow/NumPy functions

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## Type Conversions

All data in a Tensor need to be compatible, this means all need to be the same data type.

In [17]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: ignored

In [18]:
tf.constant(2.) + tf.constant(40, dtype=tf.float64)

InvalidArgumentError: ignored

We can use `tf.cast()` when we really need to convert types.

In [19]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## Variables

In [21]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [22]:
v.assign( 2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [24]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [26]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [27]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

# Customizing Models and Training Algorithms

## Custom Loss Functions

Create a Hubber Loss function

In [28]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

We can use this loss function to compile a Keras model.

```py
model.compile(loss=huber_fn, optimizer='ndam')
model.fit(X_train, y_train, [...])
```

## Saving and Loading Models That Contain Custom Components

When we load a model with a custom function, we need to map the name to the objects:

```
model = keras.models.load_model('my_model.h5', custom_objects={'hubber_fn': hubber_fn)
```